In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

In [2]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [3]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [4]:
cols = df.columns

In [5]:
numerical = []
categorical = []

In [6]:
for col in cols:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('NA')
        
        if col != 'converted':
            categorical.append(col)
    else:
        df[col] = df[col].fillna(0)
        
        if col != 'converted':
            numerical.append(col)

In [7]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [8]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
#Question 1

In [10]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [11]:
#Question 2

In [12]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [13]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [14]:
y_train = df_train['converted']
y_val = df_val['converted']
y_test = df_test['converted']

In [15]:
df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
#Question 3

In [17]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_train.converted)
 
mi = df_train[categorical].apply(mutual_info_converted_score)
round(mi, 2)

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

In [18]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [19]:
#Question 4

In [20]:
from sklearn.feature_extraction import DictVectorizer
 
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
 
dv.fit(train_dicts)
X_train = dv.transform(train_dicts)
# instead of last two lines, you can also use
# X_train = dv.fit_transform(train_dicts)
 
X_train.shape

(876, 31)

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
df_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98
...,...,...,...,...,...,...,...,...
871,organic_search,other,1,43907.0,employed,australia,4,0.33
872,social_media,retail,3,64969.0,employed,north_america,1,0.18
873,NA,education,3,89042.0,employed,asia,4,0.75
874,social_media,manufacturing,1,0.0,self_employed,europe,1,0.65


In [23]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [24]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [25]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [26]:
y_pred = model.predict_proba(X_val)[:,1]

In [27]:
converted_decision = y_pred > 0.5
converted_decision

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,

In [28]:
round((y_val == converted_decision).mean(), 2)

np.float64(0.7)

In [29]:
original_accuracty = (y_val == converted_decision).mean()

In [30]:
#Question 5

In [31]:
my_cols = list(cols)

In [32]:
my_cols.remove('converted')

In [33]:
my_cols

['lead_source',
 'industry',
 'number_of_courses_viewed',
 'annual_income',
 'employment_status',
 'location',
 'interaction_count',
 'lead_score']

In [35]:
for col in my_cols:
    features = [feature for feature in categorical + numerical if feature != col]
    train_dicts = df_train[features].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dicts)
    X_train = dv.transform(train_dicts)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dicts = df_val[features].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    y_pred = model.predict_proba(X_val)[:,1]
    converted_decision = y_pred > 0.5
    accuracy = (y_val == converted_decision).mean()
    print(f"Remove column {col}, Accuracy: {accuracy}, Diff: {original_accuracty - accuracy}")

Remove column lead_source, Accuracy: 0.7030716723549488, Diff: -0.0034129692832765013
Remove column industry, Accuracy: 0.6996587030716723, Diff: 0.0
Remove column number_of_courses_viewed, Accuracy: 0.5563139931740614, Diff: 0.14334470989761094
Remove column annual_income, Accuracy: 0.8532423208191127, Diff: -0.15358361774744034
Remove column employment_status, Accuracy: 0.6962457337883959, Diff: 0.0034129692832763903
Remove column location, Accuracy: 0.7098976109215017, Diff: -0.010238907849829393
Remove column interaction_count, Accuracy: 0.5563139931740614, Diff: 0.14334470989761094
Remove column lead_score, Accuracy: 0.7064846416382252, Diff: -0.0068259385665528916


In [ ]:
#Question 6

In [40]:
for C in [0.01, 0.1, 1, 10, 100]:
    features = categorical + numerical
    train_dicts = df_train[features].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dicts)
    X_train = dv.transform(train_dicts)
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    val_dicts = df_val[features].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    y_pred = model.predict_proba(X_val)[:,1]
    converted_decision = y_pred > 0.5
    accuracy = (y_val == converted_decision).mean()
    print(f"With C = {C}, Accuracy: {round(accuracy, 3)}")

With C = 0.01, Accuracy: 0.7
With C = 0.1, Accuracy: 0.7
With C = 1, Accuracy: 0.7
With C = 10, Accuracy: 0.7
With C = 100, Accuracy: 0.7
